This notebook documents the data aquisistion. Results are gathered in `raw_measurements.hdf5`.

In [ ]:
%load_ext autoreload
%autoreload 2

import librosa
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import h5py
from pathlib import Path
from tqdm.notebook import tqdm

import os
import sys
sys.path.insert(0, os.path.abspath(os.path.dirname(os.getcwd())))  # add src module to path
from src.measurement import (_post_process_signal, exponential_sweep, pink_noise, multitone, playrec)
from src.util import  raw_encode

T = 10  # excitation duration
sr = 96000  # sample rate 
nperseg = 2**14  # window length

def volt_to_quant(x, shunt_resistance=0.5, voltage_devider=5, velocity_sensitivity=1):
  """Convert line level to measurement quantities."""
  return x * np.array([voltage_devider, velocity_sensitivity, 1/shunt_resistance])

def make_signals():
  np.random.seed(42)
  f_end = 10000
  t = np.arange(T*sr)/sr
  sigs = [
    ("zeros", None, np.zeros(sr*T)),
    ("sweepup", None, exponential_sweep(T, sr, f_end=f_end, rms=1, fade=0.5, post_silence=0.5)),
    ("mtone", *multitone(T, sr, octave_fraction=1/3, f_end=f_end, nperseg=nperseg, rms = 1)),
    ("pink10-10k_1", None, pink_noise(T, sr, bandpass=(10, f_end), rms=1, fade=0.01, post_silence=0.1)),
    ("pink10-10k_2", None, pink_noise(T, sr, bandpass=(10, f_end), rms=1, fade=0.01, post_silence=0.1)),
    ("way_down", None, _post_process_signal(librosa.load("../data/samples/way_down_deep.wav",
                                                        sr=sr, duration=T)[0], 
                                                        sr, rms=1, fade=0.01, post_silence=0.1)),
  ]
  return sigs

sigs = make_signals()

measurement_folder = Path("../data")
db_measurements_path = measurement_folder / "raw_measurements.hdf5"
db_excitation_path = measurement_folder / "excitations.hdf5"

# start fresh
with h5py.File(db_excitation_path, "w") as f:
  for (name, aux, u) in sigs:
    sig_grp = f.create_group(name)
    sig_grp['u'] = u
    sig_grp.attrs['sr'] = sr
    sig_grp['aux_encoded'] = raw_encode(aux)

All signals we will test

In [ ]:
def signal_summary(sigs, window='boxcar'):
  flim = (20, sr/2)
  nsig = len(sigs)
  fig, ax = plt.subplots(nrows=nsig, ncols=3, sharex='col', figsize=(10, nsig*1))
  for i, (name, freqs, x) in enumerate(sigs):
    f, s = sig.welch(x, sr, window=window, nperseg=nperseg, detrend=False)
    ax[i, 0].plot(f, 10*np.log10(s)+1e-16)
    ax[i, 0].set_ylabel(name, rotation='horizontal', ha='right', size=14)
    ax[i, 0].set_xscale("log")
    ax[i, 0].set_ylim(-100, 3)
    ax[i, 0].set_xlim(flim)
    f, t, S = sig.spectrogram(x, sr, nperseg=nperseg, window=window, nfft=nperseg)
    ax[i, 1].pcolormesh(f[1:], t, 10*np.log10(S[1:].T+1e-16), vmax=0, vmin=-100, rasterized=True)
    ax[i, 1].set_xscale("log")
    ax[i, 1].set_xlim(flim)
    ax[i, 2].hist(x, bins=100, density=True)

    ax[i, 0].tick_params(bottom=False, left=False)
    ax[i, 1].tick_params(bottom=False, left=False)
    ax[i, 2].tick_params(bottom=False, left=False)

    if i != len(sigs) - 1:
      ax[i, 0].set(yticklabels=[], xticklabels=[])
      ax[i, 1].set(yticklabels=[], xticklabels=[])
      ax[i, 2].set(yticklabels=[], xticklabels=[])

  ax[-1, 0].tick_params(bottom=True, left=False)
  ax[-1, 1].tick_params(bottom=True, left=False)
  ax[-1, 2].tick_params(bottom=True, left=False)

  ax[0, 0].set_title("Spectrum")
  ax[0, 1].set_title("Spectrogram")
  ax[0, 2].set_title("Histogram")

signal_summary(sigs, 'boxcar')
plt.tight_layout()
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.05, hspace=0.05)

Measure noise floor

In [ ]:
T = 5
indata = np.zeros(sr*T)
outdata = volt_to_quant(playrec(indata, sr))
t = np.arange(len(outdata))/sr
plt.plot(t, outdata)
plt.xlabel('Time [s]')
plt.legend(["Voltage after devider [V]", "Velocity [m/s]", "Current through shunt [A]"])

Check polarity (Velocity and current should start in positive direction)

In [ ]:
T = 0.5
t = np.arange(int(T*sr))/sr
f = 0.5 * 1/T
indata = 0.1*np.sin(2*np.pi*f*t)
outdata = volt_to_quant(playrec(indata, sr))
plt.plot(t, outdata/np.max(outdata, axis=0))
plt.xlabel('Time [s]')
plt.ylabel('Quant a.u.')
plt.legend(["Voltage after devider", "Velocity", "Current through shunt"])

Measure amplifier gain

In [ ]:
T = 1
navg = 1
t = np.arange(0, T*sr) / sr
f = 1000
win = sig.windows.hann(len(t))
signal = np.sin(2*np.pi*f*t)
signal = np.random.normal(size=len(t))
signal *= np.hanning(len(signal))

amps = np.repeat(np.arange(0.01, 0.4, 0.04), navg)
np.random.shuffle(amps)  # NOTE: NI card starting is makes some funny results
amps = np.concatenate(([0], amps))
outrms = np.zeros(amps.size)
inrms = np.zeros(amps.size) 
gains = np.zeros(amps.size)
for i, amp in enumerate(amps):
    outdata = amp * signal     
    indata = volt_to_quant(playrec(outdata, sr))[:, 0]
    outrms[i] = np.std(outdata)
    inrms[i] = np.std(indata)
    gains[i] = inrms[i]/outrms[i]
    print("amp:", amp, "gain:", gains[i])

meas_amp_gain = np.mean(gains[1:])
amp_gain = 22
print("meas amp_gain", meas_amp_gain)
print("fix amp_gain", amp_gain)


In [ ]:
plt.figure()
plt.plot(outrms[1:], inrms[1:], 'x')
plt.xlabel('Digital RMS')
plt.ylabel('Voltage RMS');

plt.figure()
plt.plot(amps[1:], gains[1:], 'x')  # Note, NI card starting is makes some funny results
plt.xlabel('Digital RMS')
plt.ylabel('Gain');

Test rms range

In [ ]:
max_rms_volts = 4
levels = 1/np.array([1, 2, 4, 8])
gains = [max_rms_volts * level / amp_gain for level in levels]

T = 3
u_amp_est = pink_noise(T, sr, bandpass=(10, 10000), rms=1, fade=0.01)

for rms in max_rms_volts * levels[:1]:
    gain = rms / amp_gain
    u_out = u_amp_est * gain
    indata = playrec(u_out, sr, input_mapping=['Dev2/ai0', 'Dev2/ai1', 'Dev2/ai2'])
    u, v, i = volt_to_quant(indata).T
    power = (u*i*1/sr).sum()/T
    print(f"u_rms: {u.std():.2f} V, i_rms: {i.std():.2f} A, power: {power:.2f} W, NI Vmax: {np.max(np.abs(indata), axis=0)}")

Measurement

In [ ]:
from pathlib import Path
from time import sleep
from datetime import datetime

pause_between_measurements = 1
num_reps = 5
driver_name = '3LBR-B004-04'
dset_opt = dict(dtype=np.float32, compression='gzip')

with (h5py.File(db_excitation_path, "r") as db_excitations,
      h5py.File(db_measurements_path, "a") as db_measurements):

  # save some metadata do measurement
  meas_ls_grp = db_measurements.create_group(driver_name)
  
  # FINISH HERE WITH ALL THESE GROUPS
  zeros_done = False
  for sig_name in tqdm(db_excitations):
    u_out_before_gain = db_excitations[sig_name]['u']
    num_samples = len(u_out_before_gain)
    num_levels = len(levels)

    meas_ls_sig_grp = meas_ls_grp.create_group(sig_name)
    meas_ls_sig_grp.create_dataset('u_out_before_gain', data=u_out_before_gain, **dset_opt)
    meas_ls_sig_grp.attrs['levels'] = levels
    meas_ls_sig_grp.attrs['gains'] = gains
    meas_ls_sig_grp.attrs['max_rms_volts'] = max_rms_volts
    meas_ls_sig_grp.attrs['pause_between_measurements'] = pause_between_measurements
    meas_ls_sig_grp.attrs['amp_gain'] = amp_gain
    meas_ls_sig_grp.attrs['meas_amp_gain'] = meas_amp_gain
    meas_ls_sig_grp.attrs['datetime'] = datetime.now().isoformat()

    if sig_name != 'zeros':
      shape = (3, num_samples, num_levels, num_reps)
    else:
      shape = (3, num_samples, 1, 1)
    dset = meas_ls_sig_grp.create_dataset('rec', shape=shape, **dset_opt)

    gains = [max_rms_volts * level / amp_gain for level in levels]
    for l, (level, gain) in enumerate(zip(levels, gains)):
      u_out = u_out_before_gain * gain
      for r in tqdm(range(num_reps), leave=False):
        # playback and record
        indata = playrec(u_out, sr, input_mapping=['Dev2/ai0', 'Dev2/ai1', 'Dev2/ai2'])
        u, v, i = volt_to_quant(indata).T
        dset[:, :, l, r] = np.stack((u, i, v), axis=0)
        print(f"u_rms: {u.std():.2f} V, i_rms: {i.std():.2f} A, power: {(u*i*1/sr).sum()/T:.2f} W, NI Vmax: {np.max(np.abs(indata), axis=0)}")
        if sig_name == 'zeros': break
      if sig_name == 'zeros': break
          
      print(f"recorded {sig_name} at level {level}")
      sleep(pause_between_measurements)
    